# Saving Algerian Constitution 2020 into SurrealDB

## 1. Importing libraries

In [10]:
%pip install surrealist einops openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [11]:
from datetime import datetime
from surrealist import Surreal
from openai import OpenAI
import json
import os

%reload_ext dotenv
%dotenv .env

## 2. Load the data

In [12]:
json_file_path = os.path.join("json", "dz_consti_2020.json")
with open(json_file_path, "r", encoding="utf-8") as f:
    constitution_data = json.load(f)

## 3. Connect to the database

In [14]:
surreal = Surreal("http://127.0.0.1:28900", credentials=(os.getenv("SURREAL_USER"), os.getenv("SURREAL_PASS"))) 
print(surreal.is_ready())
print(surreal.version())

True
surrealdb-2.1.2


## 4. Embed articles and save to database

In [15]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"

def save_constitution():
    with surreal.connect() as db:
        db.use('rag_course', "test")
        db.query("DEFINE TABLE articles")
        titles = constitution_data['titles'][1:]
        for title in titles:
            chapters = title['chapters']
            for chapter in chapters:
                articles = chapter['articles']
                for article in articles:
                    # more about this on https://platform.openai.com/docs/guides/embeddings#how-to-get-embeddings
                    response = client.embeddings.create(
                        input=f"{title['header']}: {chapter['header']}: {article}",
                        model=embed_model_name,
                    )
                    vector = response.data[0].embedding

                    db.insert(
                        "articles",
                        [{
                            "text": article,
                            "embed_vector": vector,
                            "chapter": chapter['header'],
                            "title": title['header'],
                            "edition": datetime(2020, 1, 1).strftime('%Y-%m-%dT%H:%M:%SZ'),
                        }]
                    )


save_constitution()

## 5. Test Search in Database

In [16]:
user_question = "worshiping in Algeria"
# user_question = "Qu'est-ce que je peux faire dans la guerre?"

response = client.embeddings.create(
    input=user_question,
    model=embed_model_name,
)
q_vector = response.data[0].embedding

with surreal.connect() as db:
    db.use('rag_course', "test")
    res = db.query("SELECT text, vector::similarity::cosine(embed_vector, $q_vector) AS sim FROM articles ORDER BY sim DESC LIMIT 10", {"q_vector": q_vector})

print(json.dumps(res.result, indent=2, ensure_ascii=False))

[
  {
    "sim": 0.21518109895288523,
    "text": "المادة 2 : الإسلام  دين الدّولة."
  },
  {
    "sim": 0.19594855762608876,
    "text": "المادة 42  : لا مساس بحُرمة حرّيّة المعتقَد، وحُرمة حرّيّة الرّأي. حرية ممارسة العبادة مضمونة في ظل احترام القانون."
  },
  {
    "sim": 0.1955959529057602,
    "text": "المادة 76  : على كلّ مواطن أن يؤدّي بإخلاص واجباته تجاه المجموعة الوطنيّة. التزام المواطن إزاء الوطن وإجباريّة المشاركة في الدّفاع عنه، واجبان مقدّسان دائمان. تضمن الدّولة احترام رموز الثّورة، وأرواح الشّهداء، وكرامة ذويهم، والمجاهدين. وتعمل كذلك على ترقية كتابة التاريخ وتعليمه للأجيال الناشئة."
  },
  {
    "sim": 0.19093915343717585,
    "text": "المادة 10 : لا يجوز للمؤسّسات أن تقوم بما يأتي : - الممارسات الإقطاعيّة، والجهويّة، والمحسوبيّة، - إقامة علاقات الاستغلال والتّبعيّة، - السّلوك المخالف للخُلُق الإسلاميّ وقيم ثورة نوفمبر."
  },
  {
    "sim": 0.18562842007122676,
    "text": "المادة 30 : الجزائر متضامنة مع جميع الشّعوب الّتي تكافح من أجل التّحرّر السّياسيّ والاقتصاديّ، وا